In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten
using Random, Distributions
Random.seed!(5)

MersenneTwister(UInt32[0x00000005], Random.DSFMT.DSFMT_state(Int32[-85655800, 1072952617, -522203688, 1073721902, 1588056292, 1072823262, 1930843968, 1073164255, -719121749, 1073022455  …  647343590, 1073165527, 1405924594, 1072769931, 1969171087, 42945320, -226019479, -1469955508, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000000

In [2]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")
include("decoder_utils.jl")

collectygold (generic function with 1 method)

In [3]:
d = JLD.load("english_chmodel3.jld2")
d["new_d2"]["unkchar"] = Char(0x11)
d["new_d2"]["sowchar"] = Char(0x12)
d["new_d2"]["eowchar"] = Char(0x13)

'\x13': ASCII/Unicode U+0013 (category Cc: Other, control)

In [4]:
v = create_vocab(d["new_d2"])
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d["new_d2"])

9-element Array{KnetArray{Float32,2},1}:
 KFloat32[0.593419 0.190279 … 0.11003 -0.000173569; -0.236287 0.455614 … 0.138825 -0.0971059; … ; -0.341316 0.0835031 … -0.0285478 -0.0718531; -0.205958 -0.168918 … -0.13226 -0.0319474]
 KFloat32[0.0708071 -0.456333 … 0.557484 0.094816; -1.61625 -1.37924 … 0.342474 0.548784; … ; 1.07664 -0.430524 … -0.360356 0.668587; 0.324491 -0.175173 … -0.790751 1.16417]            
 KFloat32[1.2185; 1.52402; … ; -1.1745; -1.53404]                                                                                                                                        
 KFloat32[1.47578 0.055072 … -0.2823 -1.58338; -1.95253 -0.402249 … -0.221281 -0.667834; … ; 1.28302 -0.0459206 … -2.12551 -0.95058; 0.924036 0.269601 … 0.515548 7.38111]               
 KFloat32[-0.916012; -2.25621; … ; -0.166855; 0.169325]                                                                                                                                  
 KFloat32[1.70814 0.787587 … 

In [5]:
ev = extend_vocab!(v, corpus)

ExtendedVocab(Dict("Abr"=>26,"PCNom"=>18,"Dup"=>27,"PCIns"=>30,"Since"=>32,"Postp"=>31,"Interj"=>24,"Ques"=>14,"Demons"=>5,"NAdj"=>11…), Dict("VerbForm=Vnoun"=>43,"Person=3"=>4,"Case=Acc"=>11,"Number=Sing"=>7,"Mood=Nec"=>54,"Aspect=Prog"=>27,"Person[psor]=3"=>13,"VerbForm=Conv"=>44,"Aspect=Hab"=>15,"Voice=CauPass"=>46…), Vocab(Dict('E'=>45,'«'=>94,'⁄'=>110,']'=>109,'Z'=>78,'B'=>37,'5'=>67,'h'=>20,'i'=>17,'é'=>115…), Dict{String,Int64}(), Dict("null"=>8668,"Secure"=>7932,"Puppy"=>9040,"progression"=>11757,"gathered"=>6954,"Core"=>3600,"lovers"=>8832,"underground"=>9093,"backup"=>5047,"caught"=>2501…), "<s>", "</s>", "<unk>", '\x12', '\x13', '\x11', Dict("NUM"=>0x09,"DET"=>0x06,"X"=>0x11,"VERB"=>0x10,"PROPN"=>0x0c,"ADV"=>0x03,"ADJ"=>0x01,"AUX"=>0x04,"PUNCT"=>0x0d,"PRON"=>0x0b…), Dict("reparandum"=>0x23,"root"=>0x01,"vocative"=>0x24,"csubj"=>0x0f,"advmod"=>0x04,"expl"=>0x14,"iobj"=>0x18,"mark"=>0x1a,"advcl"=>0x03,"nmod"=>0x1b…)))

In [6]:
fs = createcolvecs(corpus,ev)

FeatureSource([-0.542853 -0.992666 … 1.6983 -2.17757; 0.584793 1.14789 … 0.609551 -2.53964; … ; -0.825722 -2.10416 … 1.05649 -0.0918268; -2.90343 -0.18003 … -0.493267 1.06719], [1.46431 -0.123083 … 0.537261 0.982212; 0.260871 0.318797 … 0.798075 0.462872; … ; 1.29726 0.789473 … -0.589126 0.00480883; -1.61916 1.94446 … -1.57499 -1.55228], [0.819346 0.699932 … -0.915596 -1.791; 0.504342 -2.51908 … -0.0451559 0.201541; … ; 0.493422 0.875163 … -0.962536 1.63154; 0.395833 0.817469 … -1.82224 0.441632], [-0.748226 -0.922036 … 0.198586 0.439687; 0.0337717 -0.819915 … -0.632879 0.89832; … ; 0.602362 0.948544 … 1.0591 0.150626; -2.37601 -0.228753 … 0.66468 -0.153997])

In [7]:
fillvecs!(wmodel, corpus, v, fs)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{KnetArray{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:63
└ @ Main /home/ec2-user/GBDP/encoder_utils.jl:63


In [8]:
fillcavec!(corpus)


In [9]:
function non_lin(w,x)
   return transpose(w[1:length(w)-1])*x .+ w[length(w)] 
end

non_lin (generic function with 1 method)

In [10]:
batches = minibatch(corpus, 10)

369-element Array{Any,1}:
 Any[.. , .. , ... , . , . , ... , Kanamıyordu ... , Gülümsemeler . , Niye ? , Tiyatrocular . ]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [199]:
# Going into autograd

In [11]:
function mynll(y,a::AbstractArray{<:Integer}; dims=1, average=true)
    indices = Knet.findindices(y,a,dims=dims)
    for i in 1:length(a)
       if (a[i]==0)
           deleteat!(indices, i) 
            break
        end
    end
    lp = logp(y,dims=dims)[indices]
    average ? -mean(lp) : -sum(lp)
end

mynll (generic function with 1 method)

In [12]:
# ******
struct Linear; w; b; end
(m::Linear)(x) = sigm.(m.w * x .+ m.b)
(m::Linear)(x, y) = mynll(reshape(m(x), (length(y), length(y))), y)
#(m::Linear)(data::Data) = mean(m(x,y) for (x,y) in data)
Linear(i::Int,o::Int,scale=0.01) = Linear(Param(KnetArray(map(Float32,scale * randn(o,i)))), Param(KnetArray(map(Float32,zeros(o)))))

Linear

In [13]:
# ***
model = Linear(length(corpus[1].cavec[1]) * 2, 1)

Linear(P(KnetArray{Float32,2}(1,1900)), P(KnetArray{Float32,1}(1)))

In [203]:
createrelmatrix(s)

1900×49 KnetArray{Float32,2}:
 -7.41375e-37  -7.41375e-37  -7.41375e-37  …   0.764993      0.764993   
 -1.73598e-8   -1.73598e-8   -1.73598e-8       9.42393e-10   9.42393e-10
  0.00176632    0.00176632    0.00176632       0.00138876    0.00138876 
  1.13315e-17   1.13315e-17   1.13315e-17     -4.5141e-10   -4.5141e-10 
 -4.05578e-10  -4.05578e-10  -4.05578e-10      1.47003e-6    1.47003e-6 
 -1.27773e-6   -1.27773e-6   -1.27773e-6   …   0.761234      0.761234   
  0.999986      0.999986      0.999986         2.28465e-19   2.28465e-19
  0.992126      0.992126      0.992126        -0.000163521  -0.000163521
 -8.91426e-13  -8.91426e-13  -8.91426e-13      0.952176      0.952176   
  3.66729e-6    3.66729e-6    3.66729e-6      -6.969e-9     -6.969e-9   
  4.42818e-11   4.42818e-11   4.42818e-11  …  -0.761648     -0.761648   
  0.937807      0.937807      0.937807        -0.00152425   -0.00152425 
 -3.11614e-9   -3.11614e-9   -3.11614e-9       8.32032e-7    8.32032e-7 
  ⋮                  

In [205]:
model(createrelmatrix(s))

1×49 KnetArray{Float32,2}:
 0.527146  0.498921  0.532305  0.529347  …  0.54238  0.532601  0.525596

In [14]:
function sgdupdate!(func, args; lr=0.0005)
    fval = @diff func(args...)
    for param in Knet.params(fval)
        ∇param = grad(fval, param)
        param .-= lr * ∇param
    end
    return value(fval)
end

sgdupdate! (generic function with 1 method)

In [15]:
epoch = 20

20

In [16]:
for e in 1:epoch
   for s in corpus #b
        if length(s)>2
            println(s.word)
            rm = createrelmatrix(s)
            println(size(rm))
            loss = sgdupdate!(model, (rm, s.head))
            println(loss) 
        end
    end 
    #sleep(10)
end

["Sanal", "parçacıklar", "sa", "bunların", "hiçbirini", "yapamazlar", "."]
(1900, 49)
1.949883
["Ona", "her", "şeyimi", "verdim", "."]
(1900, 25)
1.6082518
["Karşısında", ",", "pantolonu", "dizlerine", "dek", "ıslak", ",", "önlük", "torbası", "ham", "eriklerle", "dolu", "İbrahim", "dikiliyordu", "."]
(1900, 225)
2.7076275
["Nereden", "biliyorsunuz", "."]
(1900, 9)
1.1038924
["Aradığımı", "Buldum", "Sandım", "."]
(1900, 16)
1.3868431
["Kahveleri", "bende", "içelim", "."]
(1900, 16)
1.3888633
["Ne", "diyeceğimi", "bilemedim", "."]
(1900, 16)
1.3844087
["Süpürgen", "nerede", "."]
(1900, 9)
1.0999439
["Bir", "taksi", "bulduk", "."]
(1900, 16)
1.3731487
["Mebrure", "birden", "heyecanlanmıştı", "."]
(1900, 16)
1.374707
["\"", "Burada", "ki", "üst", "düzey", "görüşmelerimizde", "turizm", "için", "ellerinden", "geleni", "yapacaklarını", "söylediler", "."]
(1900, 169)
2.5628843
["Gel", "bak", ",", "sana", "evi", "göstereyim", "."]
(1900, 49)
1.9426098
["Kendileriyle", "görüştüğümden", "edindiği

(1900, 25)
1.5885358
["Hele", "o", "zamanlar", "moda", "olan", "sizler", "sözcüğüne", "ben", "hala", "şaşarım", "."]
(1900, 121)
2.3941047
["Yattığı", "yerde", ",", "kumların", "üzerinde", "gerinebildiği", "kadar", "gerindi", "."]
(1900, 81)
2.165751
["Raporları", "detay", "lı", "incelemek", "için", "se", "zamanla", "yarışacaklardı", "..."]
(1900, 81)
2.1907992
["Almanca", "ve", "İngilizce'yi", "çok", "iyi", "konuşan", "Jouni", ",", "sık", "sık", "yurtdışı", "seferlerine", "çıkıyordu", "."]
(1900, 196)
2.6290915
["70'lere", "kadar", "gidebildim", "."]
(1900, 16)
1.3706267
["Çok", "isabet", "buyurdunuz", "."]
(1900, 16)
1.3866239
["İddia", "lı", "oldukça", "değer", "kazandıklarını", "düşünüyorlar", "."]
(1900, 49)
1.9354156
["Kız", "sabah", "olunca", "rüyasını", "hatırlamış", "."]
(1900, 36)
1.7742062
["Mebrure", "yanıbaşımda", "titriyordu", "."]
(1900, 16)
1.3603677
["Korku", "dolu", "ydum", "."]
(1900, 16)
1.3650727
["Ne", "iş", "yaptığımızı", "falan", "filan", "."]
(1900, 36)
1.77935

2.3742046
["Dört", "külah", "dondurmayı", "iki", "eliyle", "tutan", "kız", "takunyalarını", "tıkırdatarak", "içeri", "girdi", "."]
(1900, 144)
2.4733393
["Ecevit", "partisinin", "bakanlarıyla", "protokolün", "bir", "tarafında", "otururken", ",", "diğer", "tarafta", "CHP", "'li", "ler", "yer", "aldı", "."]
(1900, 256)
2.7619128
["Oysa", ",", "bunları", "kaçırmaya", "hiç", "niyeti", "yok", "tu", "."]
(1900, 81)
2.1516128
["Galiba", "haklı", "."]
(1900, 9)
1.0204558
["Bize", "uygun", "düşüyor", "."]
(1900, 16)
1.3468282
["Birkaç", "adım", "atıp", ",", "kumsalı", "okşayan", "yumuşak", "dalgaların", "çıplak", "ayaklarına", "değmesini", "bekledi", "."]
(1900, 169)
2.540944
["Konduramadım", ",", "teşhirci", "değil", "."]
(1900, 25)
1.5937314
["Tahılcı", "Anadolu", "insanı", ",", "maydanozu", "azaltıp", "bulguru", "çoğaltmış", "."]
(1900, 81)
2.1778238
["Bir", "mikrofon", ",", "ağzının", "tam", "karşısında", ",", "bir", "mikrofon", "da", "Recep'in", "darbukasının", "önünde", "dururdu", "."]
(1

(1900, 400)
2.9831367
["İyi", "ki", "o", "zamanlar", "Milliyetçi", "Cephe", "icat", "edilmemiş", "."]
(1900, 81)
2.1870065
["Pay", "çıkaracaksın", "kendine", "."]
(1900, 16)
1.3799381
["Ne", "kadar", "ürkütücü", "ve", "bir", "o", "kadar", "da", "çekici", "bir", "yer", "orası", "."]
(1900, 169)
2.6004424
["Hele", "bunlar", "erkek", "ve", "kadın", "sa", "..."]
(1900, 49)
1.9327581
["Önlerinde", "hazırlanan", "raporların", "hepsi", "var", "dı", "artık", "ve", "zaman", "makinesi", "bir", "kez", "daha", ",", "bu", "sefer", "geriye", ",", "kazanın", "olduğu", "o", "ilk", "anlara", "doğru", "çalıştı", ".."]
(1900, 676)
3.2457778
["Stas", ",", "evin", "dışında", "ki", "yine", "kütüklerden", "inşa", "edilmiş", "depoda", "yeni", "avlanmış", "bir", "yaban", "koyununun", "derisini", "ve", "boynuzunu", "gösteriyor", "misafire", "."]
(1900, 441)
3.0145414
["Ne", "okulu", "oğlum", ",", "daha", "erken", "."]
(1900, 49)
1.9032946
["Şimdi", "varırız", "oraya", ",", "dedi", "Kerem", "."]
(1900, 49)
1.901

(1900, 841)
3.350531
["Bence", "de", "adı", "Süpermen", "olsun", "."]
(1900, 36)
1.739209
["Taşhan'ın", "kapısını", "itip", "bu", "yarı", "karanlık", "dünyanın", "içine", "girdim", "."]
(1900, 100)
2.2661607
["Sevimli", "de", "bir", "sebze", "dir", "."]
(1900, 36)
1.7440233
["Zor", "şey", "oraya", "yeniden", "girebilmek", "."]
(1900, 36)
1.7127959
["Tabii", "bu", "onlarda", "da", "var", "."]
(1900, 36)
1.7387352
["Sezonu", "kapamakta", "inatçı", "bir", "genç", "sürat", "motoruyla", "göremediğim", "birine", "fiyakalar", "yapıyordu", "daireler", "çizerek", "."]
(1900, 196)
2.6181026
["Bizi", "kıyaslayan", ",", "karşılaştıranlar", "bilsinler", "ki", "...", "Biz", ",", "yarışta", "değiliz", "."]
(1900, 144)
2.4332604
["Baksanıza", "hallerine", "...", "diyerek", "Mahmut'un", "masasını", "işaret", "etti", "."]
(1900, 81)
2.1826406
["Seben", "Gölü'ne", "ulaşmıştık", "artık", "."]
(1900, 25)
1.5711956
["Ancak", "bu", "olaylar", "dizisi", "nedensellik", "ilkesi", "bağlamında", "değil", ",", "ta

...

(1900, 81)
2.1175873
["Eşi", "geri", "döndükten", "sonra", "da", "bir", "süre", "İstanbul'da", "kalmayı", "planlayan", "genç", "kadın", ",", "aniden", "fikir", "değiştirip", "dönmeye", "karar", "verdi", "."]
(1900, 400)
2.9085486
["Buldum", "işte", "."]
(1900, 9)
0.9431232
["Bu", "da", "gösteriyor", "ki", ",", "uçak", "pist", "yerine", "toprağa", "vurmuştur", "\"", "diye", "konuştu", "."]
(1900, 196)
2.4152536
["Sadece", "donuk", "bakışlarla", "etrafını", "süzüyor", "."]
(1900, 36)
1.5403413
["Evden", "mi", "kaçtın", "?", "..."]
(1900, 25)
1.3434391
["Uçağın", "gövde", "(", "tekne", ")", "sigortasının", "tüm", "riskinin", "Türk", "Hava", "Yolları", "(", "THY", ")", "tarafından", "yabancı", "reasürans", "şirketlerine", "sigorta", "ettirildiğini", "bildiren", "Ray", "Sigorta", "Genel", "Müdürü", "Ercüment", "Çetin", "Alanya", ",", "yolcu", "mali", "mesuliyeti", "sigortalarının", "kendilerinde", "olduğunu", "söyledi", "."]
(1900, 1369)
3.512315
["Nasıl", "bir", "kadın", "o", "?", "diye", 

["Bu", "atılımın", "son", "dalgaları", ",", "sonuna", "ulaşmakta", "olduğumuz", "yüzyılın", "ilk", "yarısında", ",", "fizikte", "izafiyet", "ve", "kuantum", "teorileriyle", ",", "biyolojide", "genetik", "alanında", "gerçekleşmiştir", "."]
(1900, 529)
2.9414742
["Beyin", "Takımı", "diye", "isim", "edindiler", "."]
(1900, 36)
1.67526
["Zeynep", ":", "Giyecek", ",", "yiyecek", "götürelim", "."]
(1900, 49)
1.6773636
["Keskin", "toplum", "eleştirisi", ",", "espri", "li", "ve", "zeki", "özeleştiri", ",", "bir", "çeşit", "kara", "mizah", ",", "bilgi", "düzeylerinin", "çok", "üstünde", "bir", "entelektüel", "performans", "onlarla", "tanışanları", "şaşırtır", "ve", "çarpar", "."]
(1900, 784)
3.2003462
["Viski", "bardağını", "aldı", ",", "dikip", "bitirdi", "."]
(1900, 49)
1.6946454
["Ağabeyim", "hep", "CHP'den", "yana", "."]
(1900, 25)
1.5678501
["Bu", "ülkenin", "önde", "gelen", "gazetelerinin", "özel", "yayın", "ilkeleri", "arasında", "çok", "azının", "konuya", "yer", "ayırdığı", "anlaşılıyor

(1900, 324)
2.771426
["Göstergelerde", "bir", "anormallik", "oldu", "."]
(1900, 25)
1.3279619
["yedi", "yıl", "baba", "olma", "hasretiyle", "yanan", "Murat", "Öğretmen", ",", "ikibuçuk", "ay", "önce", "dünyaya", "gelen", "kızı", "İrem", "ve", "eşi", "Eser'i", "kazada", "kaybetti", "."]
(1900, 484)
3.0564053
["Eşi", "ve", "kızı", "öldü", "ama", "o", "bilmiyor", "."]
(1900, 64)
1.8755497
["Uçaktan", "indiğimde", ",", "Üstünde", "kiler", "olmaz", "."]
(1900, 49)
1.7186912
["-", "Okuyorum", "."]
(1900, 9)
1.0040209
["Kuantum", "kuramının", "öngörüsü", "olan", "sanal", "parçacıklar", "metafizik", "kavram", "lardır", "(", ")", "."]
(1900, 144)
2.3436902
["Bu", "rahat", "davranış", "lı", "New", "York", "lu", "kız", ",", "tedirginlikleri", "büyütmeye", "elverişli", "içimi", "sakinleştirmişti", "."]
(1900, 225)
2.5478873
["Bilim", "tarihinde", ",", "çok", "uzun", "ve", "zahmet", "li", "bir", "kolektif", "sürecin", "ürünü", "olan", "mantığı", "saymazsak", ",", "iki", "önem", "li", "teorik", "atı

Excessive output truncated after 524288 bytes.


(1900, 169)
2.403465
["Sen", "de", "fena", "değilsin", "."]
(1900, 25)
1.4622521
["Sözgelimi", "Zeynep", ",", "hiç", "de", "sandığım", "gibi", "vurdumduymaz", "bir", "anne", "değilmiş", "."]
(1900, 144)
2.5322564
["Herkes", "bir", "şey", "yakıştırıyor", "."]
(1900, 25)
1.3403699
["Kemal'in", "sözlerini", "tekrarlayarak", ",", "sevgilinin", "uyurken", "ki", "beş", "ayrı", "halini", "yirmisekizer", ",", "Hülya'nın", "gözkapaklarına", "çöküp", "toplanan", "buğunun", "renklerini", "otuzbeşer", "defa", "gözlerimin", "önünden", "geçirdikten", "sonra", ",", "uyuyamayacağımı", "anlayıp", "doğruldum", "."]
(1900, 841)
3.235252
["Buz", "kesiyorum", "gitgide", ",", "aldırdığı", "yok", "."]
(1900, 49)
1.9884336
["Kolayca", "eşelenen", "kuru", "karın", "altında", "ki", "otlar", ",", "kış", "ayları", "boyunca", "yeterli", "gıdayı", "sağlıyordu", "onlara", "."]
(1900, 256)
2.614315
["Ama", "asıl", "avuçları", "iyi", "tanıyordu", "o", "cetveli", "."]
(1900, 64)
1.8893273
["Hayır", ",", "dedi", "Kerem

In [17]:
s = corpus[1]

Sanal parçacıklar sa bunların hiçbirini yapamazlar . 

In [18]:
# ********* silmeeee
accuracy(reshape(model(createrelmatrix(s)), (length(s),length(s))), s.head)

0.42857142857142855

In [19]:
function ygoldroot(sentence)
        s=length(sentence)
        lambda_gold=zeros(s+1,s+1)
        for j=1:s
            lambda_gold[sentence.head[j]+1,j+1]=1.0
        end
    return lambda_gold
end

ygoldroot (generic function with 1 method)

In [20]:
ypred = reshape(model(createrelmatrix(s)),(length(s),length(s)))

7×7 KnetArray{Float32,2}:
 0.0611836   0.0586954   0.119949    …  0.0399614   0.0562006   0.0102903  
 0.963479    0.961892    0.982198       0.943977    0.960167    0.808019   
 0.0819291   0.0786679   0.157281       0.0539244   0.0753922   0.0140375  
 0.908143    0.904392    0.953867       0.863285    0.900332    0.611993   
 0.85245     0.846808    0.923563       0.786781    0.840734    0.479631   
 0.992789    0.992466    0.996539    …  0.988756    0.992114    0.956456   
 0.00185279  0.00177289  0.00386707     0.00118417  0.00169318  0.000296053

In [21]:
function ypredroot(ypred,heads) # heads = s.head, ypred n x n matrix
    padded = vcat(KnetArray(map(Float32,zeros(1,length(heads)+1))),hcat(KnetArray(map(Float32,zeros(length(heads),1))),ypred))
    index = findall(i->i == 0, heads)[1] # ASSUMES ROOT ARC(ZERO) NO MORE THAN ONE TIME
    padded[1,index+1] = 1
    return padded
end

ypredroot (generic function with 1 method)

In [22]:
ypredroott = ypredroot(ypred,s.head)

8×8 KnetArray{Float32,2}:
 0.0  0.0         0.0         …  0.0         1.0         0.0        
 0.0  0.0611836   0.0586954      0.0399614   0.0562006   0.0102903  
 0.0  0.963479    0.961892       0.943977    0.960167    0.808019   
 0.0  0.0819291   0.0786679      0.0539244   0.0753922   0.0140375  
 0.0  0.908143    0.904392       0.863285    0.900332    0.611993   
 0.0  0.85245     0.846808    …  0.786781    0.840734    0.479631   
 0.0  0.992789    0.992466       0.988756    0.992114    0.956456   
 0.0  0.00185279  0.00177289     0.00118417  0.00169318  0.000296053

In [23]:
function Eisner(n, lambda)
    #n=length(S)
    #E=Array{Any}(n,n,2,2)
    #=for s=1:n
        for d=1:2 c=1:2
            E[s,s,d,c]=0
        end
    end=#
    E=zeros(n+1,n+1,2,2)
    A=fill(Set(),(n+1,n+1,2,2))
    #A=Array{Any}(n,n,2,2)
    #temp_scores=zeros(2,2)
    #temp_sets=fill(Set(),(2,2))
    for m=1:n+1
        for s=1:n+1
            t=s+m
            t>n+1 && break
            max_score=max_q=-1
            for q=s:t-1
                tree_score=E[s,q,2,1]+E[q+1,t,1,1]+lambda[t,s] # line 20
                if tree_score>max_score
                    max_score=tree_score
                    max_q=q
                end
            end
            #temp_scores[1,2]=max_score
            E[s,t,1,2]=max_score
            if max_q>0
                #temp_sets[1,2]=union(A[s,max_q,2,1],A[max_q+1,t,1,1],Set([(t,s)]))
                A[s,t,1,2]=union(A[s,max_q,2,1],A[max_q+1,t,1,1],Set([(t-1,s-1)]))
            #else 
                #A[s,t,1,2]=union(A[s,s,2,1],A[s+1,t,1,1])
            end
            max_score=max_q=-1
            for q=s:t-1
                tree_score=E[s,q,2,1]+E[q+1,t,1,1]+lambda[s,t]
                if tree_score>max_score
                    max_score=tree_score
                    max_q=q
                end
            end
            #temp_scores[2,2]=max_score
            E[s,t,2,2]=max_score
            if max_q>0
                #temp_sets[2,2]=union(A[s,max_q,2,1],A[max_q+1,t,1,1],Set([(s,t)]))
                A[s,t,2,2]=union(A[s,max_q,2,1],A[max_q+1,t,1,1],Set([(s-1,t-1)]))
            #else
                #A[s,t,2,2]=union(A[s,s,2,1],A[s+1,t,1,1])
            end
            max_score=max_q=-1
            for q=s:t-1
                tree_score=E[s,q,1,1]+E[q,t,1,2]
                if tree_score>max_score
                    max_score=tree_score
                    max_q=q
                end
            end
            #temp_scores[1,1]=max_score
            E[s,t,1,1]=max_score
            if max_q>0
                #temp_sets[1,1]=union(A[s,max_q,1,1],A[max_q,t,1,2])
                A[s,t,1,1]=union(A[s,max_q,1,1],A[max_q,t,1,2])
            end
            max_score=max_q=-1
            for q=s+1:t
                tree_score=E[s,q,2,2]+E[q,t,2,1]
                if tree_score>max_score
                    max_score=tree_score
                    max_q=q
                end
            end
            #temp_scores[2,1]=max_score
            E[s,t,2,1]=max_score
            if max_q>0
                #temp_sets[2,1]=union(A[s,max_q,2,2],A[max_q,t,2,1])
                A[s,t,2,1]=union(A[s,max_q,2,2],A[max_q,t,2,1])
            end
            #=for i=1:2 j=1:2
               E[s,t,i,j]=temp_scores[i,j]
               A[s,t,i,j]=temp_sets[i,j]
            end=#
        end
    end
    #=dependents=ones(n)
    for pair in A[1,n,2,1]
        dependents[pair[2]]=0
    end
    push!(A[1,n,2,1],(0,findfirst(dependents)))=#
    E[1,n+1,2,1],A[1,n+1,2,1]
end

Eisner (generic function with 1 method)

In [24]:
function eisner(lambda)
    headsize = size(lambda,2)-1
    heads = Array{Int}(undef,headsize)
    score, arcset = Eisner(size(lambda,2)-1, lambda)
    for arc in arcset
        heads[arc[2]]=arc[1]
    end
    heads
end

eisner (generic function with 1 method)

In [25]:
eisnerheads = eisner(ypredroott)

7-element Array{Int64,1}:
 6
 6
 6
 6
 6
 0
 6

In [26]:
function findmaxscores(ypred) # n+1 x n+1 scores matrix
    tuples = map(Tuple,argmax(Array(ypred), dims=1))
    indices = []
    for i in tuples
        push!(indices,i[1]) 
    end
    return indices
end

findmaxscores (generic function with 1 method)

In [27]:
maxheads = findmaxscores(ypredroott)

8-element Array{Any,1}:
 1
 7
 7
 7
 7
 7
 1
 7

In [28]:
maxheads = map(x->x-1,maxheads)[2:length(maxheads)]

7-element Array{Int64,1}:
 6
 6
 6
 6
 6
 0
 6

In [29]:
function createdeciderinput(s, maxheads, eisnerheads, deprels) # s = sentence, maxheads = argmax of predicted scores, eisnerheads = output of eisner algo
    decinput = []
    for i in 1:length(s)
        for j in 1:size(deprels,2)
            col = []
            push!(col,s.cavec[i])
            if (maxheads[i] == 0)
                push!(col, KnetArray(map(Float32,zeros(length(s.cavec[1])))))
            else
                push!(col,s.cavec[maxheads[i]])
            end

            if (eisnerheads[i] == 0)
                push!(col, KnetArray(map(Float32,zeros(length(s.cavec[1])))))
            else
                push!(col,s.cavec[eisnerheads[i]])
            end

            push!(col, KnetArray(map(Float32, deprels[:,j])))
            
            push!(decinput,vcat(col...))
            
        end
    end
    return decinput = hcat(decinput...)
end

createdeciderinput (generic function with 1 method)

In [30]:
# ******
struct Linear2; w; b; end
(m::Linear2)(x) = sigm.(m.w * x .+ m.b)
(m::Linear2)(x, y) = mynll(reshape(m(x), (size(fs.deprels,2), length(y))), y; dims=1)
#(m::Linear)(data::Data) = mean(m(x,y) for (x,y) in data)
Linear2(i::Int,o::Int,scale=0.01) = Linear2(Param(KnetArray(map(Float32,scale * randn(o,i)))), Param(KnetArray(map(Float32,zeros(o)))))

Linear2

In [31]:
decidermodel = Linear2(length(corpus[1].cavec[1]) * 3 + tagrepsize * 100, 1)

Linear2(P(KnetArray{Float32,2}(1,6050)), P(KnetArray{Float32,1}(1)))

In [32]:
deciderinput = createdeciderinput(s, maxheads, eisnerheads, fs.deprels)

6050×259 KnetArray{Float32,2}:
 -7.41375e-37  -7.41375e-37  -7.41375e-37  …   0.764993      0.764993   
 -1.73598e-8   -1.73598e-8   -1.73598e-8       9.42393e-10   9.42393e-10
  0.00176632    0.00176632    0.00176632       0.00138876    0.00138876 
  1.13315e-17   1.13315e-17   1.13315e-17     -4.5141e-10   -4.5141e-10 
 -4.05578e-10  -4.05578e-10  -4.05578e-10      1.47003e-6    1.47003e-6 
 -1.27773e-6   -1.27773e-6   -1.27773e-6   …   0.761234      0.761234   
  0.999986      0.999986      0.999986         2.28465e-19   2.28465e-19
  0.992126      0.992126      0.992126        -0.000163521  -0.000163521
 -8.91426e-13  -8.91426e-13  -8.91426e-13      0.952176      0.952176   
  3.66729e-6    3.66729e-6    3.66729e-6      -6.969e-9     -6.969e-9   
  4.42818e-11   4.42818e-11   4.42818e-11  …  -0.761648     -0.761648   
  0.937807      0.937807      0.937807        -0.00152425   -0.00152425 
 -3.11614e-9   -3.11614e-9   -3.11614e-9       8.32032e-7    8.32032e-7 
  ⋮                 

In [ ]:
reshape(decidermodel(deciderinput), (37,7))

In [36]:
decidermodel(deciderinput, s.deprel)

3.6910834f0

In [37]:
#for e in 1:epoch
   for s in corpus #b
        if length(s)>2
            println(s.word)
            ypred = ypredroot(reshape(model(createrelmatrix(s)),(length(s),length(s))),s.head)
            maxheads = findmaxscores(ypred)
            maxheads = map(x->x-1,maxheads)[2:length(maxheads)] #refactor so that root excluded
            eisnerheads = eisner(ypred)
            di = createdeciderinput(s, maxheads, eisnerheads, fs.deprels)
            println(size(di))
            loss = sgdupdate!(decidermodel, (di, s.deprel))
            println(loss) 
        end
    end 
    #sleep(10)
#end

["Sanal", "parçacıklar", "sa", "bunların", "hiçbirini", "yapamazlar", "."]
(6050, 259)
3.6910834
["Ona", "her", "şeyimi", "verdim", "."]
(6050, 185)
3.7101886
["Karşısında", ",", "pantolonu", "dizlerine", "dek", "ıslak", ",", "önlük", "torbası", "ham", "eriklerle", "dolu", "İbrahim", "dikiliyordu", "."]
(6050, 555)
3.66346
["Nereden", "biliyorsunuz", "."]
(6050, 111)
3.7076633
["Aradığımı", "Buldum", "Sandım", "."]
(6050, 148)
3.7188993
["Kahveleri", "bende", "içelim", "."]
(6050, 148)
3.6621194
["Ne", "diyeceğimi", "bilemedim", "."]
(6050, 148)
3.6634555
["Süpürgen", "nerede", "."]
(6050, 111)
3.658092
["Bir", "taksi", "bulduk", "."]
(6050, 148)
3.6281428
["Mebrure", "birden", "heyecanlanmıştı", "."]
(6050, 148)
3.60921
["\"", "Burada", "ki", "üst", "düzey", "görüşmelerimizde", "turizm", "için", "ellerinden", "geleni", "yapacaklarını", "söylediler", "."]
(6050, 481)
3.6210961
["Gel", "bak", ",", "sana", "evi", "göstereyim", "."]
(6050, 259)
3.673256
["Kendileriyle", "görüştüğümden", "

(6050, 148)
3.3565798
["Siyah", "rugan", "."]
(6050, 111)
3.3247802
["Bu", ",", "tersinir", "bir", "süreç", "tir", "."]
(6050, 259)
3.416655
["Evet", ",", "ilk", "kocası", "."]
(6050, 185)
3.400248
["Hele", "o", "zamanlar", "moda", "olan", "sizler", "sözcüğüne", "ben", "hala", "şaşarım", "."]
(6050, 407)
3.406153
["Yattığı", "yerde", ",", "kumların", "üzerinde", "gerinebildiği", "kadar", "gerindi", "."]
(6050, 333)
3.4428601
["Raporları", "detay", "lı", "incelemek", "için", "se", "zamanla", "yarışacaklardı", "..."]
(6050, 333)
3.382542
["Almanca", "ve", "İngilizce'yi", "çok", "iyi", "konuşan", "Jouni", ",", "sık", "sık", "yurtdışı", "seferlerine", "çıkıyordu", "."]
(6050, 518)
3.453572
["70'lere", "kadar", "gidebildim", "."]
(6050, 148)
3.4302015
["Çok", "isabet", "buyurdunuz", "."]
(6050, 148)
3.3415506
["İddia", "lı", "oldukça", "değer", "kazandıklarını", "düşünüyorlar", "."]
(6050, 259)
3.3497436
["Kız", "sabah", "olunca", "rüyasını", "hatırlamış", "."]
(6050, 222)
3.3133838
["Mebru

(6050, 333)
3.2826042
["İşte", "onun", "ruhu", "böyle", "."]
(6050, 185)
3.2032146
["Sana", "açmayacağıma", "söz", "vermiştim", "..."]
(6050, 185)
3.2010427
["Pantolon", "imalatıyla", "ilgili", "bütün", "tasarılar", "gerçek", "oldu", "."]
(6050, 296)
3.2370412
["Evlilik", "aşkı", "desek", "!", ".."]
(6050, 185)
3.1369855
["Park", "kurşun", "sesleri", "ile", "yankılanmış", "."]
(6050, 222)
3.2238433
["Eski", "sevgilisi", "kapının", "dışında", "bekliyor", "."]
(6050, 222)
3.2210274
["Her", "yanım", "titriyor", ",", "ter", "içinde", "kaldım", "."]
(6050, 296)
3.272109
["Tek", "hesabın", "kapsamı", "genişliyor", "."]
(6050, 185)
3.1819286
["Kız", "içeri", "girmedi", "."]
(6050, 148)
3.1918917
["Ben", "tarafsız", "."]
(6050, 111)
3.1318498
["Göklere", "çıkarıyordun", "."]
(6050, 111)
3.175557
["Halbuki", "şu", "anda", "işyerlerinin", "sağladığı", "finansmanla", "bu", "iş", "yürüyor", "\"", "dedi", "."]
(6050, 444)
3.3324144
["Ve", "fakat", "bu", "yeni", "gerçek", ",", "Kadıköy", "vapurunun"

(6050, 185)
3.2206204
["Evenlerin", "tarihte", "dokuz", "devlet", "kurduğunu", "ve", "bunlardan", "dokuz.", "yüzyılda", "kurulan", "Bahai", "ile", "oniki.", "yüzyılda", "kurulan", "Sin'in", "(", "Altın", ")", "imparatorluk", "olduğunu", "da", "belirtti", "."]
(6050, 888)
3.3260932
["Her", "gün", "yüz", "bin", "mi", "verecekler", "sana", "."]
(6050, 296)
3.2923222
["Hadi", "bakalım", "düşelim", "yola", ".", "dedi", "babam", "."]
(6050, 296)
3.2717714
["Siz", "gene", "de", "demeyin", "."]
(6050, 185)
3.1798608
["Mahmut", "Beyi", "aramıştım", "."]
(6050, 148)
3.2599328
["Biraz", "da", "elleri", "sıkı", "dır", "."]
(6050, 222)
3.266821
["J", ",", "ziyareti", "sırasında", "Türkiye", "ekonomisinin", "genel", "durumu", "üzerinde", "de", "duracaklarını", "ifade", "etti", "."]
(6050, 518)
3.1632621
["Beni", "sınava", "mı", "çekiyorsunuz", "?"]
(6050, 185)
3.2848618
["\"", "Bu", "sözleri", "ağlayarak", "dinleyen", "Özmen", ",", "programında", "çalışması", "için", "Ciguli'ye", "teklifte", "bulunu

(6050, 740)
3.282463
["Bütün", "bunlara", "rağmen", ",", "Sav", "familyası", "imajını", "zedelemeyerek", "aynı", "okulları", "bitirdim", "ben", "de", "."]
(6050, 518)
3.1831274
["'", "seçim", "döneminde", "kendisini", "Çankaya", "Köşkü'ne", "çıkaracağı", "yolunda", "Milli", "Görüş", "tabanına", "propaganda", "yaptığını", "öne", "süren", "kapatılan", "RP'nin", "Genel", "Başkanı", "Necmettin", "Erbakan'a", "'", "milletvekilleri", "sert", "tepki", "gösterdi", "."]
(6050, 999)
3.250615
["Ağzının", "kenarında", "var", "mıydı", "bu", "derin", "çizgi", "?"]
(6050, 296)
3.2845492
["Patlıcan", ",", "deyimlerimize", "de", "girmiştir", "."]
(6050, 222)
3.0993323
["Bu", "da", "bölgede", "ki", "ticarete", "katkıda", "bulunacak", "."]
(6050, 296)
3.1679192
["Durun", ",", "oturun", "biraz", "."]
(6050, 185)
3.122213
["Kadın", "uyuduysa", "okur", ",", "dedi", "adam", "."]
(6050, 259)
3.0879953
["Bilimsel", "Devrim", "boyunca", "alanını", "din", "aleyhine", "gittikçe", "genişleterek", ",", "dinin", "es

...

(6050, 333)
3.13129
["Bartın'a", "yeni", "tayin", "oldu", "."]
(6050, 185)
3.0432465
["home", "lideri", "Recep", "Tayyip", "Erdoğan'ı", "taşıyan", "uçağın", "Pekin'e", "zorunlu", "iniş", "yapacağının", "duyulmasının", "ardından", "olayı", "Türkiye'ye", "bildirmek", "isteyen", "gazetecilerle", "işadamları", "arasında", "kısa", "süre", "li", "bir", "gerginlik", "yaşandı", "."]
(6050, 999)
3.1918132
["Aldatma", "üstüne", "kurulmuş", "bir", "mekanizma", "."]
(6050, 222)
3.2122087
["Hakkı", ",", "Seni", "bu", "kafayla", "bekar", "evine", "göndermem", "."]
(6050, 333)
3.0522625
["Birinci", ",", "Ayhan", "'dı", "."]
(6050, 185)
3.1886277
["Abajuru", "fişe", "takınca", "ışık", "verdi", "."]
(6050, 222)
2.998206
["Gecekondu", "yapmak", "için", "para", "biriktiriyor", "."]
(6050, 222)
3.085649
["Mahmut", ",", "dedi", "."]
(6050, 148)
2.9908721
["Gırtlağıma", "kadar", "dolu", "yum", "demiş", "."]
(6050, 222)
3.3941846
["Havaalanından", "yaklaşık", "otuz", "kişiyi", "aramış", ",", "herkesle", "şak

(6050, 296)
3.1474576
["Sonra", ",", "patron", "olduğunu", "anımsatmak", "için", ",", "soymakta", "olduğu", "patatesleri", "bırakıp", ",", "bir", "çay", "doldurdu", ",", "masalardan", "birine", "geçti", ",", "sabahtan", "beri", "bakıla", "bakıla", "paçavra", "gibi", "olmuş", "gazeteyi", "eline", "aldı", "."]
(6050, 1147)
3.1614773
["Çok", "yıllık", "bütçe", "yapılsın", "."]
(6050, 185)
3.0959077
["Bir", "çay", "daha", "..."]
(6050, 148)
3.1464534
["Komşuların", "kapılarını", "teker", "teker", "çalıp", "iş", "istemiş", "."]
(6050, 296)
3.0552707
["On", "sekiz", "yaşında", "sırasıyla", ",", "içki", ",", "kokain", "ve", "eroinle", "tanışıyor", "."]
(6050, 444)
3.2597313
["Naci", "Bey", ",", "şu", "sayfadan", "şu", "sayfaya", "kadar", "okuyun", ",", "demiş", "."]
(6050, 444)
3.1024427
["Makine", "ile", "benim", "aramda", ",", "dedim", "."]
(6050, 259)
3.1452944
["Daha", "doğrusu", "takipsizlik", "kararı", "verilmişti", "."]
(6050, 222)
3.0665996
["Ama", "onu", "bir", "daha", "gören", "olma

(6050, 1147)
3.182588
["Demiray", ",", "\"", "Çünkü", "kredi", "kartını", "çıkaran", "kuruluş", "bu", "işi", "kendisi", "finanse", "ederek", "faiz", "ve", "komisyon", "almak", "ister", "."]
(6050, 703)
3.2132187
["İyice", "hasta", "ydı", ",", "diyor", ",", "karnı", "davul", "gibi", "şişmişti", "."]
(6050, 407)
3.2298994
["Uçları", "yukarı", "kıvrık", "sipsivri", "bıyıkları", "vardı", "babasının", "."]
(6050, 296)
3.023573
["Başkalarının", "en", "iyi", "gizlenen", "sırlarına", "bile", "ulaşmanın", "daima", "bir", "yolu", "bulunurdu", ",", "ama", "insanın", "kendi", "sırlarına", "erişmesi", "neredeyse", "imkansız", "dı", "ve", "yazarlık", "bu", "imkansızı", "yaşamak", "tı", "sürekli", "."]
(6050, 1036)
3.2323744
["paralel", "bağlanabiliyor", ";", "ama", "bir", "trafik", "kazasında", "insanların", "paralel", "bağlanabileceğini", "ilk", "kez", "duyuyorum", "."]
(6050, 518)
3.1405675
["Tam", "o", "sırada", "Hülya", "birisiyle", "gelebilir", "ve", "beni", "görebilir", "diye", "..."]
(6050, 4

(6050, 518)
3.1304884
["Sermaye", "yapısı", "değişti", "."]
(6050, 148)
2.99125
["Ağzını", "yokluyor", "bir", "yandan", "."]
(6050, 185)
3.087464
["Sürünür", "şimdi", "."]
(6050, 111)
3.0242367
["Ben", "de", "o", "gece", "kardeşlerimin", "yatma", "saati", "gelene", "kadar", "uzun", "uzun", "köpeğimizi", "anlattım", "onlara", ",", "ne", "de", "olsa", "o", "artık", "evimizin", "köpeği", "ydi", "."]
(6050, 888)
3.1680927
["Sevinmez", "mi", "?"]
(6050, 111)
3.3301349
["Bir", "dilim", "ekmeğe", "muhtaç", "ız", ",", "diye", "yanıtlamış", "küçük", "kız", "."]
(6050, 407)
3.2219918
["Bu", "da", "ayıplar", "arasında", "."]
(6050, 185)
3.012713
["Eve", "geldiklerinde", "de", "çok", "yorgun", "durlar", "veya", "yapacakları", "bir", "şeyler", "var", "dır", "."]
(6050, 481)
3.3946924
["Anadolu'da", "zor", "olmalı", "kız", "-", "erkek", "ilişkileri", "."]
(6050, 296)
3.0212593
["Kapı", "hafifçe", "aralandı", "."]
(6050, 148)
3.0201445
["Yağma", "yok", "diye", "diklendi", "Memo", ",", "hepiniz", "hav

(6050, 555)
3.0706806
["Hastaneye", "yatırmışlar", "kaç", "defa", ",", "biraz", "düzelir", "gibi", "oluyor", ",", "ay", "geçmeden", "başlıyor", "daha", "beter", "içmeye", "."]
(6050, 629)
3.118025
["Hızlı", "hızlı", "yürümeye", "başlamıştı", "."]
(6050, 185)
3.049124
["Esin'in", "bu", "cevabımı", "çok", "beğendiği", "her", "halinden", "belli", "oluyordu", "."]
(6050, 370)
3.0932462
["Kavurma", "ustalığından", "."]
(6050, 111)
2.9915166
["Sıkılmış", "kravatların", "ve", "dar", "pabuçların", "esaretinde", "boğulmadan", "."]
(6050, 296)
3.2588463
["Bağımlı", "kişinin", "beyninde", "duygular", "eşit", "tir", "acı", "çekmek", "formülü", "var", "dır", "."]
(6050, 444)
3.1810627
["Elbet", "ayrımında", "lar", "."]
(6050, 148)
3.2332447
["Bence", "zamanın", "geçmesi", "tehlike", "li", "."]
(6050, 222)
3.0640278
["Tabii", "bir", "ayağım", "Ankara'da", "idi", "."]
(6050, 222)
3.2249985
["Ben", "se", "korkuyordum", "."]
(6050, 148)
3.4556122
["Hep", "senin", "yüzünden", "."]
(6050, 148)
3.0123928


In [42]:
function fillparsers!(corpus, deprels)
    for s in corpus
            ypred = model(createrelmatrix(s))
            ypred = ypredroot(reshape(ypred, (length(s),length(s))),s.head)
            maxheads = findmaxscores(ypred)
            maxheads = map(x->x-1,maxheads)[2:length(maxheads)] #refactor so that root excluded
            eisnerheads = eisner(ypred)
            di = createdeciderinput(s, maxheads, eisnerheads, deprels)
        s.parse = Parser(eisner(ypred), findmaxscores(reshape(decidermodel(di), (size(deprels,2),length(s)) )))
    end
end

fillparsers! (generic function with 1 method)

In [43]:
fillparsers!(corpus, fs.deprels)

In [44]:
s.parse.deprel

7-element Array{UInt8,1}:
 0x22
 0x22
 0x22
 0x22
 0x22
 0x22
 0x22

In [45]:
function las(corpus)
    nword = ncorr = 0
    for s in corpus
        p = s.parse
        nword += length(s)
        ncorr += sum((s.head .== p.head) .& (s.deprel .== p.deprel))
    end
    ncorr / nword
end

las (generic function with 1 method)

In [46]:
las(corpus)

0.09752624083548711